### This script cleans and merges the data from the tsv files. We subset movies between 2013 to 2023, and then merge them into a consolidated dataframe

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import streamlit as st
import missingno as msno
import os
import requests
from bs4 import BeautifulSoup
import random

import sqlite3
from sqlalchemy import create_engine, MetaData, Table

from helper import *


pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns', None)

## Cleaning imdb raw files

In [ ]:

name_basics = pd.read_csv('../data/name_basics_data.tsv')
title_basics = pd.read_csv('../data/title_basics_data.tsv')
title_crew = pd.read_csv('../data/title_crew_data.tsv')
title_principals = pd.read_csv('../data/title_principals_data.tsv')
title_ratings = pd.read_csv('../data/title_ratings_data.tsv')
title_akas = pd.read_csv('../data/title_akas_data.tsv')



print(f"name_basics shape:{name_basics.shape}")
print(f"title_basics shape:{title_basics.shape}")
print(f"title_crew shape:{title_crew.shape}")
print(f"title_principals shape:{title_principals.shape}")
print(f"title_ratings shape:{title_ratings.shape}")
print(f"title_akas shape:{title_akas.shape}")

In [ ]:
# title_basics = pd.read_csv('../data/title_basics_data.tsv', sep = '\t', low_memory=False)

title_basics.replace('\\N', np.nan, inplace=True)
title_basics = title_basics.dropna(subset=['startYear'])
title_basics['startYear'] = title_basics['startYear'].astype('int')
title_basics = title_basics[(title_basics['titleType']=='movie') & (title_basics['startYear']>2013) & (title_basics['startYear']<2024)]

In [ ]:
title_basics.head()

In [ ]:
display(title_basics.info())
print("="*180)

In [ ]:
title_crew.head()

In [ ]:
display(title_crew.info())
print("="*180)

In [ ]:
title_principals.head()

In [ ]:
display(title_principals.info())
print("="*180)

In [ ]:
title_ratings.head()

In [ ]:
display(title_ratings.info())
print("="*180)

In [ ]:
name_basics.head()

In [ ]:
display(name_basics.info())
print("="*180)

In [ ]:
title_akas.head()

In [ ]:
display(title_akas.info())
print("="*180)

#### Some preliminary exploration and cleaning of each datatable first before we inject them into a database

1. title_basics

In [ ]:
# dropping rows with no genres

title_basics['startYear'] = title_basics['startYear'].astype('int')
title_basics['runtimeMinutes'] = title_basics['runtimeMinutes'].astype('float')

title_basics_gb = title_basics.groupby('startYear')['runtimeMinutes'].mean()

#Imputing runtimeMinutes na values with the mean of movies in the same year
for index, row in title_basics.iterrows():
    start_year = row['startYear']
    if pd.isnull(row['runtimeMinutes']):
        title_basics.loc[index, 'runtimeMinutes'] = title_basics_gb[start_year]

        
#Dropping endYear column
title_basics = title_basics.drop(columns=['endYear'], axis='columns')

#Dropping rows with no genres values
title_basics = title_basics.dropna(subset=['genres'])

print(title_basics.shape)

title_basics.to_csv('../data/title_basics_cleaned.csv')

2. title_principals

In [ ]:
#Dropping ordering column as it wont be required, since we are concatenating the other columns sequentially
title_principals = title_principals.drop(columns=['ordering'])

#Converting all columns to string
title_principals[['nconst', 'category', 'job', 'characters']] = title_principals[['nconst', 'category', 'job', 'characters']].astype('string')

#Removing brackets off characters column
title_principals['characters'] = title_principals['characters'].replace('[', '').replace(']','')


#Concatenating nconst, category, job and characters with ";"
title_principals = title_principals.groupby('tconst').agg(lambda x: ';'.join(x))



title_principals.to_csv('../data/title_principals_cleaned.csv')

3. title_crew

In [ ]:
title_crew.to_csv('../data/title_crew_cleaned.csv')

4. title_ratings

In [ ]:
title_ratings.to_csv('../data/title_ratings_cleaned.csv')

5. title_akas

In [ ]:
# Counting the number of regions for each titleId
region_count = title_akas.groupby('titleId')['region'].count().reset_index().rename(columns={'region':'region_count'})

# Merging the original dataframe with the region count
title_akas_cleaned= title_akas.merge(region_count, on='titleId', how='left')

title_akas_cleaned = title_akas_cleaned[title_akas_cleaned['isOriginalTitle']=='1']
title_akas_cleaned = title_akas_cleaned.drop(columns=['ordering','region','language','isOriginalTitle','types','attributes'])
title_akas_cleaned.to_csv('../data/title_akas_cleaned.csv')

6. name_basics

In [ ]:
name_basics = pd.read_csv('../data/name_basics_data.tsv', sep = '\t', low_memory=False)

name_basics = name_basics.replace('\\N',np.nan)

name_basics2 = name_basics.drop(columns=['birthYear','deathYear'])
name_basics3 = name_basics2.dropna(subset=['primaryName'])

In [ ]:

name_basics3.to_csv('../data/name_basics_cleaned.csv')

In [ ]:
#Reloading dataframes

name_basics = pd.read_csv('../data/name_basics_cleaned.csv').drop('Unnamed: 0', axis=1)
title_basics = pd.read_csv('../data/title_basics_cleaned.csv').drop('Unnamed: 0', axis=1)
title_crew = pd.read_csv('../data/title_crew_cleaned.csv').drop('Unnamed: 0', axis=1)
title_principals = pd.read_csv('../data/title_principals_cleaned.csv')
title_ratings = pd.read_csv('../data/title_ratings_cleaned.csv').drop('Unnamed: 0', axis=1)
title_akas = pd.read_csv('../data/title_akas_cleaned.csv').drop('Unnamed: 0', axis=1)

In [ ]:
df = pd.merge(left=title_basics, right=title_principals, how='left', on='tconst', sort=True)
df = pd.merge(left=df, right=title_crew, how='left', on='tconst', sort=True)
df = pd.merge(left=df, right=title_ratings, how='left', on='tconst', sort=True)
df = pd.merge(left=df, right=title_akas, how='left', left_on='tconst', right_on='titleId', sort=True)
df = pd.merge(left=df, right=revenue, on='tconst',sort=True)
df = pd.merge(left=df, right=ratings, on='tconst')


In [ ]:
df.to_csv('../data/movies2013-2023.csv')

## Cleaning ratings raw inputs

In [ ]:
#Expanding df_ratings

In [2]:
ratings = pd.read_csv('../data/df_ratings.csv')
ratings = ratings[ratings['ratings']!='[]']

ratings

,tconst,ratings
3,tt0069049,"[('ur0011762', '7'), ('ur3564718', '8'), ('ur0..."
8,tt0109128,"[('ur0808507', '9')]"
13,tt0120589,"[('ur34226907', '8')]"
14,tt0137204,"[('ur3942202', '9'), ('ur85067217', '9')]"
15,tt0137818,"[('ur128765215', '1')]"
...,...,...
174583,tt9914972,"[('ur132041895', '8')]"
174591,tt9916190,"[('ur126320906', '8'), ('ur126352070', '6'), (..."
174592,tt9916270,"[('ur93886624', '6'), ('ur24536688', '6'), ('u..."
174593,tt9916362,"[('ur82872124', '9'), ('ur126952560', '9'), ('..."


In [ ]:
ratings['ratings'].isna().sum()

In [ ]:
import ast
ratings['ratings'] = ratings['ratings'].apply(ast.literal_eval)
ratings['ratings'] = ratings['ratings'].tolist()

In [ ]:
ratings

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
for index, ratings_list in ratings['ratings'].iteritems():
    for user_id, rating in ratings_list:
        ratings.loc[index, user_id] = rating

In [ ]:
ratings.drop(columns = ['ratings'],inplace=True)

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
import scipy.sparse as sp




ratings.to_parquet('../data/df_ratings_expanded.parquet')

## Cleaning ratings2

In [9]:
ratings = pd.read_parquet('../data/df_ratings_expanded.parquet').reset_index()
ratings2 = pd.read_csv('../data/df_ratings2.csv')

In [10]:
ratings2['ratings']=ratings2['ratings'].replace(0, 'NIL')
ratings2['ratings']=ratings2['ratings'].replace('0', 'NIL')
ratings2['ratings']=ratings2['ratings'].fillna('NIL')
ratings2['ratings'] = ratings2['ratings'].astype('string')

In [11]:
ratings = ratings.set_index('tconst')
missing_values_before = ratings.isna().sum().sum()
print(f"Number of missing values in ratings:{missing_values_before}")


KeyboardInterrupt: 

In [ ]:
for ind, item in ratings2.iterrows():
    if item['ratings'] != 'NIL':
        user_id = item['user_id']
        ratings_list = ast.literal_eval(item['ratings'])
        print(f"Current index: {ind} out of {len(ratings2)}")

        for movie, rating in ratings_list:
            if (movie in ratings.index) and (user_id in ratings.columns):
                ratings.loc[movie,user_id] = rating
                print(f"Assigned movie: {movie}, user:{user_id}, rating:{ratings.loc[movie,user_id]}")

        print("="*80)

In [ ]:
missing_values_after = ratings.isna().sum().sum()
print(f"Number of missing values in ratings:{missing_values_after}")
print(f"Difference:{missing_values_before - missing_values_after}")

In [ ]:
ratings.to_parquet('../data/df_ratings_expanded.parquet')

## Exploding cast columns

In [33]:
df = pd.read_csv('../data/movies2013-2023.csv')
print(df.shape)

(163026, 22)


In [34]:
import pandas as pd

# Assuming df is your dataframe
df['nconst'] = df['nconst'].str.split(';')
df['category'] = df['category'].str.split(';')
df['job'] = df['job'].str.split(';')


# Create a new dataframe by exploding nconst, category, job
df = df.apply(lambda x: x.explode() if x.name in ['nconst', 'category', 'job'] else x)

df.reset_index(drop=True, inplace=True)

df = df.drop(columns=['Unnamed: 0'])


In [35]:
df.to_csv('../data/movies2013-2023_castexploded.csv')

## Merging name_basics with castexploded df

In [36]:
name_basics = pd.read_csv('../data/name_basics_cleaned.csv').drop(columns=['Unnamed: 0'])
name_basics

,nconst,primaryName,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,"soundtrack,actor,miscellaneous","tt0072308,tt0045537,tt0050419,tt0053137"
1,nm0000002,Lauren Bacall,"actress,soundtrack","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,"actress,soundtrack,music_department","tt0056404,tt0057345,tt0049189,tt0054452"
3,nm0000004,John Belushi,"actor,soundtrack,writer","tt0078723,tt0080455,tt0072562,tt0077975"
4,nm0000005,Ingmar Bergman,"writer,director,actor","tt0050986,tt0050976,tt0060827,tt0069467"
...,...,...,...,...
12544681,nm9993714,Romeo del Rosario,"animation_department,art_department","tt14069590,tt2455546,tt11657662"
12544682,nm9993716,Essias Loberg,NaN,NaN
12544683,nm9993717,Harikrishnan Rajan,cinematographer,tt8736744
12544684,nm9993718,Aayush Nair,cinematographer,tt8736744


In [39]:
print(f"df shape before merge: {df.shape}")
df = df.merge(name_basics, how='left', on='nconst')
print(f"df shape after merge: {df.shape}")

df shape before merge: (1198173, 21)
df shape after merge: (1198173, 24)


In [40]:
df.to_csv('../data/movies2013-2023_castexploded_mergedcastname.csv')